In [3]:
# pip install accelerate
#from transformers import T5Tokenizer, T5ForConditionalGeneration
#from openai import OpenAI

import json

In [4]:
import os
os.environ['HF_HOME']='/run/cache'

In [5]:
os.chdir(r'C:\Users\user\Documents\LLM Zoomcamp\Project')
print(os.getcwd())
file_path = os.path.join(os.getcwd(), 'data\clean_data\documents.json')
file_path


C:\Users\user\Documents\LLM Zoomcamp\Project


<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\user\AppData\Local\Temp\ipykernel_25900\1002694120.py:3: SyntaxWarning: invalid escape sequence '\c'
  file_path = os.path.join(os.getcwd(), 'data\clean_data\documents.json')


'C:\\Users\\user\\Documents\\LLM Zoomcamp\\Project\\data\\clean_data\\documents.json'

In [6]:
with open(file_path, 'r') as file:
    documents=json.load(file)
len(documents)

5837

In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")




NameError: name 'T5Tokenizer' is not defined

In [8]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

NameError: name 'OpenAI' is not defined

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
llm('how is weather in london?')

NotFoundError: Error code: 404 - {'error': {'message': 'model "phi3" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

In [20]:
x=3
y=9

In [21]:
x @ y

TypeError: unsupported operand type(s) for @: 'int' and 'int'

In [22]:
import operator

# Define matrices as lists
A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]

# Matrix multiplication
result = [[sum(a * b for a, b in zip(A_row, B_col)) for B_col in zip(*B)] for A_row in A]
print(result)


[[19, 22], [43, 50]]


In [23]:
DATA_PATH = os.getenv("DATA_PATH", "../data/clean_data/final_data_with_IDs_new.csv")


In [24]:
DATA_PATH

'data/clean_data'

In [8]:
import os
data_path=os.getenv('Data_Path')

In [26]:
os.getenv

<function os.getenv(key, default=None)>

In [34]:
os.environ['Data']='rttt'

In [35]:
os.environ['Data']


'rttt'

In [9]:
import os
#from dotenv import load_dotenv

os.environ['RUN_TIMEZONE_CHECK'] = '0'

from db import init_db

#load_dotenv()

if __name__ == "__main__":
    print("Initializing database...")
    init_db()

ModuleNotFoundError: No module named 'urlparse'

In [46]:
project=os.getenv('Project_Path')
import os
current=Path(project)
new=os.path.join(current, 'notebooks')
print(new)
os.chdir(new)
print(os.getcwd())

from pathlib import Path
import time
from openai import OpenAI
import minsearch
import json
import os
import subprocess

# Load documents
file_path = current / 'data'/ 'clean_data' / 'documents.json'
print(file_path)
#file_path = os.path.join(os.getcwd(), r'data/clean_data/documents.json')
with open(file_path, 'r') as file:
    documents = json.load(file)

# Initialize the OpenAI client
client = OpenAI()

# Initialize the minsearch index
index = minsearch.Index(
    text_fields=['Chunked_Content'],
    keyword_fields=[]
)
index.fit(documents)

def search(query):
    boost = {
        'Chunked_Content': 1.9366969407339725
    }
    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

def build_prompt(query, search_results):
    prompt_template = """
    You're a primal health adviser. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = "\n".join([f"Chunked_Content: {doc['Chunked_Content']}" for doc in search_results])
    
    return prompt_template.format(question=query, context=context).strip()

# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-3.5-turbo',
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return response.choices[0].message.content
# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return response.choices[0].message.content

def llm(prompt,model="llama3.2"):
    # Command to run Ollama with the desired model and prompt
    command = ["ollama", "run", model, prompt]

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

    # Return the result
    return result.stdout

# Example usage
response = query_ollama("What is the capital of France?")
print(response)
def rag(query):
    # Check if the user is asking who created the bot
    if query.lower() in ["who created you?", "who is your creator?", "who made you?"]:
        return "I was created by W Mohamed."

    # Otherwise, proceed with normal search and LLM response
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    time.sleep(1)  # Simulating a delay for the function to work
    return answer
    


c:\users\user\Documents\LLM Zoomcamp\Project\notebooks
c:\users\user\Documents\LLM Zoomcamp\Project\notebooks
c:\users\user\Documents\LLM Zoomcamp\Project\data\clean_data\documents.json
The capital of France is Paris.




In [44]:
query='what causes belly fat'

In [45]:
rag(query)

'Based on the CONTEXT, the question "what causes belly fat" cannot be directly answered. However, the user mentions that they have a swollen belly and experience discomfort after eating cooked food, specifically omnivorism, which implies that they may be experiencing digestive issues or intolerance to certain foods.\n\nThe user also mentions that cooking animal fat breaks it down before digestion, suggesting that perhaps raw animal fat might not be easily digestible for everyone. Additionally, the user has tried fermented foods and found them easier to digest, implying that gut health and microbiome balance might play a role in belly issues.\n\nIt is worth noting that the user does mention feeling better with long simmered meat fat or in broth, suggesting that cooking may help break down certain compounds or make nutrients more accessible. However, the exact cause of belly fat is not explicitly stated in the provided CONTEXT.\n\n'

In [ ]:
# 'Eating raw animal fat may not be as healing as expected, and cooking the fat before digestion might be necessary, especially tough fat like pork belly.
# Fermentation could be the oldest form of cooking, but dry aging and wet fermenting may have different microbial effects. High meat consumption may 
# not work for everyone, as it can lead to a nightmare experience and changes in taste. Ultimately, 
# the causes of belly fat may vary from person to person.'


In [31]:
from pathlib import Path
cwd = Path.cwd()
goal_dir = cwd.parent.parent / "my_dir"

In [32]:
cwd

WindowsPath('C:/Users/user/Documents/LLM Zoomcamp/Project')

In [33]:
goal_dir

WindowsPath('C:/Users/user/Documents/my_dir')

In [35]:
cwd.parent

WindowsPath('C:/Users/user/Documents/LLM Zoomcamp')

In [6]:
os.path.join(os.getcwd(), data_path)

NameError: name 'data_path' is not defined

In [40]:
path = Path(r'C:\Users\user\Documents\LLM Zoomcamp\Project\data\clean_data')

In [41]:
path

WindowsPath('C:/Users/user/Documents/LLM Zoomcamp/Project/data/clean_data')

In [2]:
import subprocess

def query_ollama(prompt,model="llama3.2"):
    # Command to run Ollama with the desired model and prompt
    command = ["ollama", "run", model, prompt]

    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, encoding='utf-8')

    # Return the result
    return result.stdout

# Example usage
response = query_ollama("What is the capital of France?")
print(response)


The capital of France is Paris.




In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the model name to use LLaMA model (e.g., LLaMA 7B)
model_name = "decapoda-research/llama3.2:latest"  # Example: LLaMA-7B model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Text input
input_text = "What is the capital of France?"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text using the model
outputs = model.generate(**inputs, max_length=50)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


OSError: Incorrect path_or_model_id: 'decapoda-research/llama3.2:latest'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
query='tell me about healthy food?'
def llm_flan(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt",max_length=2048, truncation=True).input_ids
    #outputs = model.generate(input_ids, max_length=3048, num_beams=4, early_stopping=True)
    outputs = model.generate(
        input_ids, 
        max_new_tokens=1024,  # Set max number of new tokens
        num_beams=4,          # Keep beams, but consider reducing for diversity
        do_sample=False,       # Use sampling instead of beam search
        top_p=0.95,            # Use nucleus sampling for diversity
        early_stopping=False,  # Allow the model to generate more tokens
        top_k=50,
        temperature=1
    )
    result=tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [1]:
# it is not working
def llm_flan(prompt, generate_params=None):
    # Specify to use CPU
    #device = torch.device("cpu")
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 1000),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [2]:
llm_flan('how to lose belly fat')

NameError: name 'tokenizer' is not defined

In [27]:
import torch
print(torch.cuda.is_available())


False


In [18]:
# # alternative method creating with adding combined
# import lancedb
# from sentence_transformers import SentenceTransformer

# # Load a pre-trained model to generate text embeddings
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # Example documents
# documents = [
#     {'id': 0, 'content': 'This is the first document.', 'number of sentences': 5, 'number of words': 56},
#     {'id': 1, 'content': 'This is the second document.', 'number of sentences': 5, 'number of words': 49}
# ]

# # Generate embeddings for the documents
# contents = [doc['content'] for doc in documents]
# embeddings = model.encode(contents)

# # Restructure the data
# data = [{"text": doc['content'], "embedding": embedding.tolist()} for doc, embedding in zip(documents, embeddings)]

# # Create or open a LanceDB database
# db = lancedb.connect("./lancedb")

# # Create the table and insert data
# table = db.create_table("my_vector_table", data=data)


In [37]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="google/gemma-7b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu",
)

messages = [
    {"role": "user", "content": "Who are you? Please, answer in pirate-speak."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-7b-it.
401 Client Error. (Request ID: Root=1-6703cb5f-0a20a0cb5d9b8f271c73ca98;e29bff85-18b5-493e-91d0-3d9645053e57)

Cannot access gated repo for url https://huggingface.co/google/gemma-7b-it/resolve/main/config.json.
Access to model google/gemma-7b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [10]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

model_path = 'openlm-research/open_llama_3b'
# model_path = 'openlm-research/open_llama_7b'

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

C:\Users\user\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\run\cache\hub\models--openlm-research--open_llama_3b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [ ]:
# mistral

from huggingface_hub import HfApi, login
import os
api_token = os.getenv("huggingface")

# Log in with your Hugging Face API token
login(token=api_token)
# Authenticate Hugging Face API
#api = HfApi()
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the Mistral 7B tokenizer and model from Hugging Face
model_name = "mistralai/Mistral-7B-v0.1"  # The model's path on Hugging Face

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Define a function to generate text using the Mistral model
def generate_text(prompt, max_new_tokens=256):
    # Encode the prompt using the tokenizer
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate tokens
    outputs = model.generate(inputs.input_ids, max_new_tokens=max_new_tokens)
    
    # Decode the tokens to generate the text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
prompt = "Tell me about the benefits of healthy food."
generated_output = generate_text(prompt, max_new_tokens=150)
print(generated_output)
